<a href="https://colab.research.google.com/github/piyush26c/Cuda-Programming/blob/main/Assignment_5_MinElem_MaxElem_Mean_StandardDeviation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [2]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-wb6by2f4
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-wb6by2f4
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=bf41d45b6f331861ce0e90ffac6bf09e37658fa145baface6a1467c97996c5f3
  Stored in directory: /tmp/pip-ephem-wheel-cache-xu6zzdkw/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
//Author : Piyush Rajendra Chaudhari
//Roll No: BECOC311
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#define VECTOR_SIZE 10

void fillVector (long long *vector_) {
	for (int indx = 0; indx < VECTOR_SIZE; indx++) {
		vector_[indx] = indx ;
	}
}

void printVector (long long *vector_) {
	for (int indx = 0; indx < VECTOR_SIZE; indx++) {
		printf("%lld ", vector_[indx]);
	}
}

__global__ void findMinElem (long long *vector_) {
	int threadId = threadIdx.x;
	long long stepSize = 1;
	int numberOfThreads = blockDim.x;
	while (numberOfThreads > 0) {
		if (threadId < numberOfThreads) {
			int firstElemIndx = threadId * stepSize * 2;
			int secondElemIndx = firstElemIndx + stepSize;
			if (vector_[firstElemIndx] > vector_[secondElemIndx]) {
				vector_[firstElemIndx] = vector_[secondElemIndx];
			}			
		}
		stepSize = stepSize * 2;
		numberOfThreads = numberOfThreads / 2;
	}
}


int main (void) {
	//program for finding minimum element from input vector
	long long *hostVector, resultMinElem;
	long long *deviceVector;
	long long memorySize = VECTOR_SIZE * sizeof(long long);
	
	// Allocate space for host vectors A and insert input values
    hostVector = (long long *)malloc(memorySize); 
	fillVector(hostVector);
	
	// Allocate space for device vectors A
    cudaMalloc((void **)&deviceVector, memorySize);
	
	// Copy vector data from host to device
    cudaMemcpy(deviceVector, hostVector, memorySize, cudaMemcpyHostToDevice);
	
	
	dim3 blocksPerGrid(1, 1, 1);
	//by creating single block with half of the size of vector threads.
	dim3 threadsPerBlock(VECTOR_SIZE / 2, 1, 1);
	findMinElem<<<blocksPerGrid, threadsPerBlock>>>(deviceVector);
	
	// Copy result back to host result variable 
    cudaMemcpy(&resultMinElem, deviceVector, sizeof(long long), cudaMemcpyDeviceToHost);
	printf("\nFinding the Minimum Element (Parallel Programming) ");
	printf("\nInput Vector : ");
	printVector(hostVector);
	printf("\nThe minimum element in given vector : %lld", resultMinElem);
	
	
	
	return 0;
}


Finding the Minimum Element (Parallel Programming) 
Input Vector : 0 1 2 3 4 5 6 7 8 9 
The minimum element in given vector : 0


In [11]:
%%cu
//Author : Piyush Rajendra Chaudhari
//Roll No: BECOC311
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#define VECTOR_SIZE (1 << (VECTOR_SIZE_))   //corresponds to 2 ^VECTOR_SIZE_
#define VECTOR_SIZE_ 5

void fillVector (long long *vector_) {
	for (int indx = 0; indx < VECTOR_SIZE; indx++) {
		vector_[indx] = indx + 1;
	}
}

void printVector (long long *vector_) {
	for (int indx = 0; indx < VECTOR_SIZE; indx++) {
		printf("%lld ", vector_[indx]);
	}
}

__global__ void findMinElem (long long *vector_) {
	int threadId = threadIdx.x;
	long long stepSize = 1;
	int numberOfThreads = blockDim.x;
	while (numberOfThreads > 0) {
		if (threadId < numberOfThreads) {
			int firstElemIndx = threadId * stepSize * 2;
			int secondElemIndx = firstElemIndx + stepSize;
			if (vector_[firstElemIndx] > vector_[secondElemIndx]) {
				vector_[firstElemIndx] = vector_[secondElemIndx];
			}			
		}
		stepSize = stepSize * 2;
		numberOfThreads = numberOfThreads / 2;
	}
}

__global__ void findMaxElem (long long *vector_) {
	
	int threadId = threadIdx.x;
	long long stepSize = 1;
	int numberOfThreads = blockDim.x;
	while (numberOfThreads > 0) {
		if (threadId < numberOfThreads) {
			int firstElemIndx = threadId * stepSize * 2;
			int secondElemIndx = firstElemIndx + stepSize;
			if (vector_[firstElemIndx] < vector_[secondElemIndx]) {
				vector_[firstElemIndx] = vector_[secondElemIndx];
			}			
		}
		stepSize = stepSize * 2;
		numberOfThreads = numberOfThreads / 2;
	}	
}



int main (void) {
	{
	//program for finding minimum element from input vector
	long long *hostVector, resultMinElem;
	long long *deviceVector;
	long long memorySize = VECTOR_SIZE * sizeof(long long);
	
	// Allocate space for host vectors A and insert input values
    hostVector = (long long *)malloc(memorySize); 
	fillVector(hostVector);
	
	// Allocate space for device vectors A
    cudaMalloc((void **)&deviceVector, memorySize);
	
	// Copy vector data from host to device
    cudaMemcpy(deviceVector, hostVector, memorySize, cudaMemcpyHostToDevice);
	
	
	dim3 blocksPerGrid(1, 1, 1);
	//by creating single block with half of the size of vector threads.
	dim3 threadsPerBlock(VECTOR_SIZE / 2, 1, 1);
	findMinElem<<<blocksPerGrid, threadsPerBlock>>>(deviceVector);
	
	// Copy result back to host result variable 
    cudaMemcpy(&resultMinElem, deviceVector, sizeof(long long), cudaMemcpyDeviceToHost);
	
	printf("\nInput Vector : ");
	printVector(hostVector);
	printf("\nThe minimum element in given vector : %lld", resultMinElem);
	}
	{
	//program for finding maximum element from input vector

	long long *hostVector, resultMaxElem;
	long long *deviceVector;
	long long memorySize = VECTOR_SIZE * sizeof(long long);
	
	// Allocate space for host vectors A and insert input values
    hostVector = (long long *)malloc(memorySize); 
	fillVector(hostVector);
	
	// Allocate space for device vectors A
    cudaMalloc((void **)&deviceVector, memorySize);
	
	// Copy vector data from host to device
    cudaMemcpy(deviceVector, hostVector, memorySize, cudaMemcpyHostToDevice);
	
	
	dim3 blocksPerGrid(1, 1, 1);
	//by creating single block with half of the size of vector threads.
	dim3 threadsPerBlock(VECTOR_SIZE / 2, 1, 1);
	findMaxElem<<<blocksPerGrid, threadsPerBlock>>>(deviceVector);
	
	// Copy result back to host result variable 
    cudaMemcpy(&resultMaxElem, deviceVector, sizeof(long long), cudaMemcpyDeviceToHost);

	printf("\nThe maximum element in given vector : %lld", resultMaxElem);
	}
	

	return 0;
}


Input Vector : 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 
The minimum element in given vector : 1
The maximum element in given vector : 32


In [27]:
%%cu
//Author : Piyush Rajendra Chaudhari
//Roll No: BECOC311
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#define VECTOR_SIZE (1 << (VECTOR_SIZE_))   //corresponds to 2 ^VECTOR_SIZE_
#define VECTOR_SIZE_ 3

void fillVector (long long *vector_) {
	for (int indx = 0; indx < VECTOR_SIZE; indx++) {
		vector_[indx] = indx + 1;
	}
}

void fillVectorWDV (double *vector_) {
	for (int indx = 0; indx < VECTOR_SIZE; indx++) {
		vector_[indx] = (double) indx + 1;
	}
}

void printVector (long long *vector_) {
	for (int indx = 0; indx < VECTOR_SIZE; indx++) {
		printf("%lld ", vector_[indx]);
	}
}

__global__ void findMinElem (long long *vector_) {
	int threadId = threadIdx.x;
	long long stepSize = 1;
	int numberOfThreads = blockDim.x;
	while (numberOfThreads > 0) {
		if (threadId < numberOfThreads) {
			int firstElemIndx = threadId * stepSize * 2;
			int secondElemIndx = firstElemIndx + stepSize;
			if (vector_[firstElemIndx] > vector_[secondElemIndx]) {
				vector_[firstElemIndx] = vector_[secondElemIndx];
			}			
		}
		stepSize = stepSize * 2;
		numberOfThreads = numberOfThreads / 2;
	}
}

__global__ void findMaxElem (long long *vector_) {
	
	int threadId = threadIdx.x;
	long long stepSize = 1;
	int numberOfThreads = blockDim.x;
	while (numberOfThreads > 0) {
		if (threadId < numberOfThreads) {
			int firstElemIndx = threadId * stepSize * 2;
			int secondElemIndx = firstElemIndx + stepSize;
			if (vector_[firstElemIndx] < vector_[secondElemIndx]) {
				vector_[firstElemIndx] = vector_[secondElemIndx];
			}			
		}
		stepSize = stepSize * 2;
		numberOfThreads = numberOfThreads / 2;
	}	
}

__global__ void findMean (double *vector_) {
	
	int threadId = threadIdx.x;
	long long stepSize = 1;
	int numberOfThreads = blockDim.x;
	while (numberOfThreads > 0) {
		if (threadId < numberOfThreads) {
			int firstElemIndx = threadId * stepSize * 2;
			int secondElemIndx = firstElemIndx + stepSize;
			vector_[firstElemIndx] += vector_[secondElemIndx];						
		}
		stepSize = stepSize * 2;
		numberOfThreads = numberOfThreads / 2;
	}
	vector_[0] = vector_[0] / (double) VECTOR_SIZE;
}


int main (void) {
	{
	//program for finding minimum element from input vector
	long long *hostVector, resultMinElem;
	long long *deviceVector;
	long long memorySize = VECTOR_SIZE * sizeof(long long);
	
	// Allocate space for host vectors A and insert input values
    hostVector = (long long *)malloc(memorySize); 
	fillVector(hostVector);
	
	// Allocate space for device vectors A
    cudaMalloc((void **)&deviceVector, memorySize);
	
	// Copy vector data from host to device
    cudaMemcpy(deviceVector, hostVector, memorySize, cudaMemcpyHostToDevice);
	
	
	dim3 blocksPerGrid(1, 1, 1);
	//by creating single block with half of the size of vector threads.
	dim3 threadsPerBlock(VECTOR_SIZE / 2, 1, 1);
	findMinElem<<<blocksPerGrid, threadsPerBlock>>>(deviceVector);
	
	// Copy result back to host result variable 
    cudaMemcpy(&resultMinElem, deviceVector, sizeof(long long), cudaMemcpyDeviceToHost);
	
	printf("\nInput Vector : ");
	printVector(hostVector);
	printf("\nThe minimum element in given vector : %lld", resultMinElem);
	}
	{
	//program for finding maximum element from input vector

	long long *hostVector, resultMaxElem;
	long long *deviceVector;
	long long memorySize = VECTOR_SIZE * sizeof(long long);
	
	// Allocate space for host vectors A and insert input values
    hostVector = (long long *)malloc(memorySize); 
	fillVector(hostVector);
	
	// Allocate space for device vectors A
    cudaMalloc((void **)&deviceVector, memorySize);
	
	// Copy vector data from host to device
    cudaMemcpy(deviceVector, hostVector, memorySize, cudaMemcpyHostToDevice);
	
	
	dim3 blocksPerGrid(1, 1, 1);
	//by creating single block with half of the size of vector threads.
	dim3 threadsPerBlock(VECTOR_SIZE / 2, 1, 1);
	findMaxElem<<<blocksPerGrid, threadsPerBlock>>>(deviceVector);
	
	// Copy result back to host result variable 
    cudaMemcpy(&resultMaxElem, deviceVector, sizeof(long long), cudaMemcpyDeviceToHost);
	
	printf("\nInput Vector : ");
	printVector(hostVector);
	printf("\nThe maximum element in given vector : %lld", resultMaxElem);
	}
	{
	//program for finding mean from input vector

	double *hostVector;
	double resultMean;
	double *deviceVector;
	double memorySize = VECTOR_SIZE * sizeof(double);
	
	// Allocate space for host vectors A and insert input values
    hostVector = (double *)malloc(memorySize); 
	fillVectorWDV(hostVector);
	
	// Allocate space for device vectors A
    cudaMalloc((void **)&deviceVector, memorySize);
	
	// Copy vector data from host to device
    cudaMemcpy(deviceVector, hostVector, memorySize, cudaMemcpyHostToDevice);
	
	
	dim3 blocksPerGrid(1, 1, 1);
	//by creating single block with half of the size of vector threads.
	dim3 threadsPerBlock(VECTOR_SIZE / 2, 1, 1);
	findMean<<<blocksPerGrid, threadsPerBlock>>>(deviceVector);
	
	// Copy result back to host result variable 
    cudaMemcpy(&resultMean, deviceVector, sizeof(double), cudaMemcpyDeviceToHost);
	
	printf("\nThe mean of the given vector : %lf", resultMean);
	}
	

	return 0;
}


Input Vector : 1 2 3 4 5 6 7 8 
The minimum element in given vector : 1
Input Vector : 1 2 3 4 5 6 7 8 
The maximum element in given vector : 8
The mean of the given vector : 4.500000


In [30]:
%%cu
//Author : Piyush Rajendra Chaudhari
//Roll No: BECOC311
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#define VECTOR_SIZE (1 << (VECTOR_SIZE_))   //corresponds to 2 ^VECTOR_SIZE_
#define VECTOR_SIZE_ 3

void fillVector (long long *vector_) {
	for (int indx = 0; indx < VECTOR_SIZE; indx++) {
		vector_[indx] = indx + 1;
	}
}

void fillVectorWDV (double *vector_) {
	for (int indx = 0; indx < VECTOR_SIZE; indx++) {
		vector_[indx] = (double) indx + 1;
	}
}

void printVector (long long *vector_) {
	for (int indx = 0; indx < VECTOR_SIZE; indx++) {
		printf("%lld ", vector_[indx]);
	}
}

__global__ void findMinElem (long long *vector_) {
	int threadId = threadIdx.x;
	long long stepSize = 1;
	int numberOfThreads = blockDim.x;
	while (numberOfThreads > 0) {
		if (threadId < numberOfThreads) {
			int firstElemIndx = threadId * stepSize * 2;
			int secondElemIndx = firstElemIndx + stepSize;
			if (vector_[firstElemIndx] > vector_[secondElemIndx]) {
				vector_[firstElemIndx] = vector_[secondElemIndx];
			}			
		}
		stepSize = stepSize * 2;
		numberOfThreads = numberOfThreads / 2;
	}
}

__global__ void findMaxElem (long long *vector_) {
	
	int threadId = threadIdx.x;
	long long stepSize = 1;
	int numberOfThreads = blockDim.x;
	while (numberOfThreads > 0) {
		if (threadId < numberOfThreads) {
			int firstElemIndx = threadId * stepSize * 2;
			int secondElemIndx = firstElemIndx + stepSize;
			if (vector_[firstElemIndx] < vector_[secondElemIndx]) {
				vector_[firstElemIndx] = vector_[secondElemIndx];
			}			
		}
		stepSize = stepSize * 2;
		numberOfThreads = numberOfThreads / 2;
	}	
}

__global__ void findMean (double *vector_) {
	
	int threadId = threadIdx.x;
	long long stepSize = 1;
	int numberOfThreads = blockDim.x;
	while (numberOfThreads > 0) {
		if (threadId < numberOfThreads) {
			int firstElemIndx = threadId * stepSize * 2;
			int secondElemIndx = firstElemIndx + stepSize;
			vector_[firstElemIndx] += vector_[secondElemIndx];						
		}
		stepSize = stepSize * 2;
		numberOfThreads = numberOfThreads / 2;
	}
	vector_[0] = vector_[0] / (double) VECTOR_SIZE;
}

__global__ void findSum (long long *vector_) {
	
	int threadId = threadIdx.x;
	long long stepSize = 1;
	int numberOfThreads = blockDim.x;
	while (numberOfThreads > 0) {
		if (threadId < numberOfThreads) {
			int firstElemIndx = threadId * stepSize * 2;
			int secondElemIndx = firstElemIndx + stepSize;
			vector_[firstElemIndx] += vector_[secondElemIndx];						
		}
		stepSize = stepSize * 2;
		numberOfThreads = numberOfThreads / 2;
	}
}



int main (void) {
	{
	//program for finding minimum element from input vector
	long long *hostVector, resultMinElem;
	long long *deviceVector;
	long long memorySize = VECTOR_SIZE * sizeof(long long);
	
	// Allocate space for host vectors A and insert input values
    hostVector = (long long *)malloc(memorySize); 
	fillVector(hostVector);
	
	// Allocate space for device vectors A
    cudaMalloc((void **)&deviceVector, memorySize);
	
	// Copy vector data from host to device
    cudaMemcpy(deviceVector, hostVector, memorySize, cudaMemcpyHostToDevice);
	
	
	dim3 blocksPerGrid(1, 1, 1);
	//by creating single block with half of the size of vector threads.
	dim3 threadsPerBlock(VECTOR_SIZE / 2, 1, 1);
	findMinElem<<<blocksPerGrid, threadsPerBlock>>>(deviceVector);
	
	// Copy result back to host result variable 
    cudaMemcpy(&resultMinElem, deviceVector, sizeof(long long), cudaMemcpyDeviceToHost);
	
	printf("\nInput Vector : ");
	printVector(hostVector);
	printf("\nThe minimum element in given vector : %lld", resultMinElem);
	}
	{
	//program for finding maximum element from input vector

	long long *hostVector, resultMaxElem;
	long long *deviceVector;
	long long memorySize = VECTOR_SIZE * sizeof(long long);
	
	// Allocate space for host vectors A and insert input values
    hostVector = (long long *)malloc(memorySize); 
	fillVector(hostVector);
	
	// Allocate space for device vectors A
    cudaMalloc((void **)&deviceVector, memorySize);
	
	// Copy vector data from host to device
    cudaMemcpy(deviceVector, hostVector, memorySize, cudaMemcpyHostToDevice);
	
	
	dim3 blocksPerGrid(1, 1, 1);
	//by creating single block with half of the size of vector threads.
	dim3 threadsPerBlock(VECTOR_SIZE / 2, 1, 1);
	findMaxElem<<<blocksPerGrid, threadsPerBlock>>>(deviceVector);
	
	// Copy result back to host result variable 
    cudaMemcpy(&resultMaxElem, deviceVector, sizeof(long long), cudaMemcpyDeviceToHost);
	
	printf("\nInput Vector : ");
	printVector(hostVector);
	printf("\nThe maximum element in given vector : %lld", resultMaxElem);
	}
	{
	//program for finding mean from input vector

	double *hostVector;
	double resultMean;
	double *deviceVector;
	double memorySize = VECTOR_SIZE * sizeof(double);
	
	// Allocate space for host vectors A and insert input values
    hostVector = (double *)malloc(memorySize); 
	fillVectorWDV(hostVector);
	
	// Allocate space for device vectors A
    cudaMalloc((void **)&deviceVector, memorySize);
	
	// Copy vector data from host to device
    cudaMemcpy(deviceVector, hostVector, memorySize, cudaMemcpyHostToDevice);
	
	
	dim3 blocksPerGrid(1, 1, 1);
	//by creating single block with half of the size of vector threads.
	dim3 threadsPerBlock(VECTOR_SIZE / 2, 1, 1);
	findMean<<<blocksPerGrid, threadsPerBlock>>>(deviceVector);
	
	// Copy result back to host result variable 
    cudaMemcpy(&resultMean, deviceVector, sizeof(double), cudaMemcpyDeviceToHost);
	
	printf("\nThe mean of the given vector : %lf", resultMean);
	}
	{
	//program for finding sum of all elements from input vector

	long long *hostVector, resultSum;
	long long *deviceVector;
	long long memorySize = VECTOR_SIZE * sizeof(long long);
	
	// Allocate space for host vectors A and insert input values
    hostVector = (long long *)malloc(memorySize); 
	fillVector(hostVector);
	
	// Allocate space for device vectors A
    cudaMalloc((void **)&deviceVector, memorySize);
	
	// Copy vector data from host to device
    cudaMemcpy(deviceVector, hostVector, memorySize, cudaMemcpyHostToDevice);
	
	
	dim3 blocksPerGrid(1, 1, 1);
	//by creating single block with half of the size of vector threads.
	dim3 threadsPerBlock(VECTOR_SIZE / 2, 1, 1);
	findSum<<<blocksPerGrid, threadsPerBlock>>>(deviceVector);
	
	// Copy result back to host result variable 
    cudaMemcpy(&resultSum, deviceVector, sizeof(long long), cudaMemcpyDeviceToHost);

	printf("\nThe summation of all the elements in given vector : %lld", resultSum);
	}
	

	return 0;
}


Input Vector : 1 2 3 4 5 6 7 8 
The minimum element in given vector : 1
Input Vector : 1 2 3 4 5 6 7 8 
The maximum element in given vector : 8
The mean of the given vector : 4.500000
The summation of all the elements in given vector : 36


In [35]:
%%cu
//Author : Piyush Rajendra Chaudhari
//Roll No: BECOC311
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#define VECTOR_SIZE (1 << (VECTOR_SIZE_))   //corresponds to 2 ^VECTOR_SIZE_
#define VECTOR_SIZE_ 5

void fillVector (long long *vector_) {
	for (int indx = 0; indx < VECTOR_SIZE; indx++) {
		vector_[indx] = indx + 1;
	}
}

void fillVectorWDV (double *vector_) {
	for (int indx = 0; indx < VECTOR_SIZE; indx++) {
		vector_[indx] = (double) indx + 1;
	}
}

void printVector (long long *vector_) {
	for (int indx = 0; indx < VECTOR_SIZE; indx++) {
		printf("%lld ", vector_[indx]);
	}
}

__global__ void findMinElem (long long *vector_) {
	int threadId = threadIdx.x;
	long long stepSize = 1;
	int numberOfThreads = blockDim.x;
	while (numberOfThreads > 0) {
		if (threadId < numberOfThreads) {
			int firstElemIndx = threadId * stepSize * 2;
			int secondElemIndx = firstElemIndx + stepSize;
			if (vector_[firstElemIndx] > vector_[secondElemIndx]) {
				vector_[firstElemIndx] = vector_[secondElemIndx];
			}			
		}
		stepSize = stepSize * 2;
		numberOfThreads = numberOfThreads / 2;
	}
}

__global__ void findMaxElem (long long *vector_) {
	
	int threadId = threadIdx.x;
	long long stepSize = 1;
	int numberOfThreads = blockDim.x;
	while (numberOfThreads > 0) {
		if (threadId < numberOfThreads) {
			int firstElemIndx = threadId * stepSize * 2;
			int secondElemIndx = firstElemIndx + stepSize;
			if (vector_[firstElemIndx] < vector_[secondElemIndx]) {
				vector_[firstElemIndx] = vector_[secondElemIndx];
			}			
		}
		stepSize = stepSize * 2;
		numberOfThreads = numberOfThreads / 2;
	}	
}

__global__ void findMean (double *vector_) {
	
	int threadId = threadIdx.x;
	long long stepSize = 1;
	int numberOfThreads = blockDim.x;
	while (numberOfThreads > 0) {
		if (threadId < numberOfThreads) {
			int firstElemIndx = threadId * stepSize * 2;
			int secondElemIndx = firstElemIndx + stepSize;
			vector_[firstElemIndx] += vector_[secondElemIndx];						
		}
		stepSize = stepSize * 2;
		numberOfThreads = numberOfThreads / 2;
	}
	vector_[0] = vector_[0] / (double) VECTOR_SIZE;
}

__global__ void findSum (long long *vector_) {
	
	int threadId = threadIdx.x;
	long long stepSize = 1;
	int numberOfThreads = blockDim.x;
	while (numberOfThreads > 0) {
		if (threadId < numberOfThreads) {
			int firstElemIndx = threadId * stepSize * 2;
			int secondElemIndx = firstElemIndx + stepSize;
			vector_[firstElemIndx] += vector_[secondElemIndx];						
		}
		stepSize = stepSize * 2;
		numberOfThreads = numberOfThreads / 2;
	}
}



int main (void) {
	{
	//program for finding minimum element from input vector
	long long *hostVector, resultMinElem;
	long long *deviceVector;
	long long memorySize = VECTOR_SIZE * sizeof(long long);
	
	// Allocate space for host vectors A and insert input values
    hostVector = (long long *)malloc(memorySize); 
	fillVector(hostVector);
	
	// Allocate space for device vectors A
    cudaMalloc((void **)&deviceVector, memorySize);
	
	// Copy vector data from host to device
    cudaMemcpy(deviceVector, hostVector, memorySize, cudaMemcpyHostToDevice);
	
	
	dim3 blocksPerGrid(1, 1, 1);
	//by creating single block with half of the size of vector threads.
	dim3 threadsPerBlock(VECTOR_SIZE / 2, 1, 1);
	findMinElem<<<blocksPerGrid, threadsPerBlock>>>(deviceVector);
	
	// Copy result back to host result variable 
    cudaMemcpy(&resultMinElem, deviceVector, sizeof(long long), cudaMemcpyDeviceToHost);
	
	printf("\nInput Vector : ");
	printVector(hostVector);
	printf("\nThe minimum element in given vector : %lld", resultMinElem);
	}
	{
	//program for finding maximum element from input vector

	long long *hostVector, resultMaxElem;
	long long *deviceVector;
	long long memorySize = VECTOR_SIZE * sizeof(long long);
	
	// Allocate space for host vectors A and insert input values
    hostVector = (long long *)malloc(memorySize); 
	fillVector(hostVector);
	
	// Allocate space for device vectors A
    cudaMalloc((void **)&deviceVector, memorySize);
	
	// Copy vector data from host to device
    cudaMemcpy(deviceVector, hostVector, memorySize, cudaMemcpyHostToDevice);
	
	
	dim3 blocksPerGrid(1, 1, 1);
	//by creating single block with half of the size of vector threads.
	dim3 threadsPerBlock(VECTOR_SIZE / 2, 1, 1);
	findMaxElem<<<blocksPerGrid, threadsPerBlock>>>(deviceVector);
	
	// Copy result back to host result variable 
    cudaMemcpy(&resultMaxElem, deviceVector, sizeof(long long), cudaMemcpyDeviceToHost);
	
	printf("\nInput Vector : ");
	printVector(hostVector);
	printf("\nThe maximum element in given vector : %lld", resultMaxElem);
	}
	{
	//program for finding mean from input vector

	double *hostVector;
	double resultMean;
	double *deviceVector;
	double memorySize = VECTOR_SIZE * sizeof(double);
	
	// Allocate space for host vectors A and insert input values
    hostVector = (double *)malloc(memorySize); 
	fillVectorWDV(hostVector);
	
	// Allocate space for device vectors A
    cudaMalloc((void **)&deviceVector, memorySize);
	
	// Copy vector data from host to device
    cudaMemcpy(deviceVector, hostVector, memorySize, cudaMemcpyHostToDevice);
	
	
	dim3 blocksPerGrid(1, 1, 1);
	//by creating single block with half of the size of vector threads.
	dim3 threadsPerBlock(VECTOR_SIZE / 2, 1, 1);
	findMean<<<blocksPerGrid, threadsPerBlock>>>(deviceVector);
	
	// Copy result back to host result variable 
    cudaMemcpy(&resultMean, deviceVector, sizeof(double), cudaMemcpyDeviceToHost);
	
	printf("\nThe mean of the given vector : %lf", resultMean);
	}
	{
	//program for finding sum of all elements from input vector

	long long *hostVector, resultSum;
	long long *deviceVector;
	long long memorySize = VECTOR_SIZE * sizeof(long long);
	
	// Allocate space for host vectors A and insert input values
    hostVector = (long long *)malloc(memorySize); 
	fillVector(hostVector);
	
	// Allocate space for device vectors A
    cudaMalloc((void **)&deviceVector, memorySize);
	
	// Copy vector data from host to device
    cudaMemcpy(deviceVector, hostVector, memorySize, cudaMemcpyHostToDevice);
	
	
	dim3 blocksPerGrid(1, 1, 1);
	//by creating single block with half of the size of vector threads.
	dim3 threadsPerBlock(VECTOR_SIZE / 2, 1, 1);
	findSum<<<blocksPerGrid, threadsPerBlock>>>(deviceVector);
	
	// Copy result back to host result variable 
    cudaMemcpy(&resultSum, deviceVector, sizeof(long long), cudaMemcpyDeviceToHost);
	
	printf("\nThe summation of all the elements in given vector : %lld", resultSum);
	}
	{
	//program for finding variance of elements from input vector
	//first find mean and then substract it from each element parallelly

	long long *hostVector, resultSum;
	long long *deviceVector;
	long long memorySize = VECTOR_SIZE * sizeof(long long);
	double mean;
	
	// Allocate space for host vectors A and insert input values
    hostVector = (long long *)malloc(memorySize); 
	fillVector(hostVector);
	
	// Allocate space for device vectors A
    cudaMalloc((void **)&deviceVector, memorySize);
	
	// Copy vector data from host to device
    cudaMemcpy(deviceVector, hostVector, memorySize, cudaMemcpyHostToDevice);
	
	
	dim3 blocksPerGrid(1, 1, 1);
	//by creating single block with half of the size of vector threads.
	dim3 threadsPerBlock(VECTOR_SIZE / 2, 1, 1);
	findSum<<<blocksPerGrid, threadsPerBlock>>>(deviceVector);
	
	// Copy result back to host result variable 
    cudaMemcpy(&resultSum, deviceVector, sizeof(long long), cudaMemcpyDeviceToHost);
	
	mean = resultSum / (double) VECTOR_SIZE;
	
	printf("\nThe summation of all the elements in given vector : %lld, %lf", resultSum, mean);
	}
	

	return 0;
}


Input Vector : 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 
The minimum element in given vector : 1
Input Vector : 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 
The maximum element in given vector : 32
The mean of the given vector : 16.500000
The summation of all the elements in given vector : 528
The summation of all the elements in given vector : 528, 16.500000


In [68]:
%%cu
//Author : Piyush Rajendra Chaudhari
//Roll No: BECOC311
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <math.h>
#define VECTOR_SIZE (1 << (VECTOR_SIZE_))   //corresponds to 2 ^VECTOR_SIZE_
#define VECTOR_SIZE_ 3

void fillVector (long long *vector_) {
	for (int indx = 0; indx < VECTOR_SIZE; indx++) {
		vector_[indx] = indx + 1;
	}
}

void fillVectorWDV (double *vector_) {
	for (int indx = 0; indx < VECTOR_SIZE; indx++) {
		vector_[indx] = (double) indx + 1;
	}
}

void printVector (long long *vector_) {
	for (int indx = 0; indx < VECTOR_SIZE; indx++) {
		printf("%lld ", vector_[indx]);
	}
}

void printVectorWDV (double *vector_) {
	for (int indx = 0; indx < VECTOR_SIZE; indx++) {
		printf("%lf ", vector_[indx]);
	}
}

__global__ void findMinElem (long long *vector_) {
	int threadId = threadIdx.x;
	long long stepSize = 1;
	int numberOfThreads = blockDim.x;
	while (numberOfThreads > 0) {
		if (threadId < numberOfThreads) {
			int firstElemIndx = threadId * stepSize * 2;
			int secondElemIndx = firstElemIndx + stepSize;
			if (vector_[firstElemIndx] > vector_[secondElemIndx]) {
				vector_[firstElemIndx] = vector_[secondElemIndx];
			}			
		}
		stepSize = stepSize * 2;
		numberOfThreads = numberOfThreads / 2;
	}
}

__global__ void findMaxElem (long long *vector_) {
	
	int threadId = threadIdx.x;
	long long stepSize = 1;
	int numberOfThreads = blockDim.x;
	while (numberOfThreads > 0) {
		if (threadId < numberOfThreads) {
			int firstElemIndx = threadId * stepSize * 2;
			int secondElemIndx = firstElemIndx + stepSize;
			if (vector_[firstElemIndx] < vector_[secondElemIndx]) {
				vector_[firstElemIndx] = vector_[secondElemIndx];
			}			
		}
		stepSize = stepSize * 2;
		numberOfThreads = numberOfThreads / 2;
	}	
}

__global__ void findMean (double *vector_) {
	
	int threadId = threadIdx.x;
	long long stepSize = 1;
	int numberOfThreads = blockDim.x;
	while (numberOfThreads > 0) {
		if (threadId < numberOfThreads) {
			int firstElemIndx = threadId * stepSize * 2;
			int secondElemIndx = firstElemIndx + stepSize;
			vector_[firstElemIndx] += vector_[secondElemIndx];						
		}
		stepSize = stepSize * 2;
		numberOfThreads = numberOfThreads / 2;
	}
	vector_[0] = vector_[0] / (double) VECTOR_SIZE;
}

__global__ void findSum (long long *vector_) {
	
	int threadId = threadIdx.x;
	long long stepSize = 1;
	int numberOfThreads = blockDim.x;
	while (numberOfThreads > 0) {
		if (threadId < numberOfThreads) {
			int firstElemIndx = threadId * stepSize * 2;
			int secondElemIndx = firstElemIndx + stepSize;
			vector_[firstElemIndx] += vector_[secondElemIndx];						
		}
		stepSize = stepSize * 2;
		numberOfThreads = numberOfThreads / 2;
	}
}

__global__ void findSumVar (double *vector_) {
	
	int threadId = threadIdx.x;
	long long stepSize = 1;
	int numberOfThreads = blockDim.x;
	while (numberOfThreads > 0) {
		if (threadId < numberOfThreads) {
			int firstElemIndx = threadId * stepSize * 2;
			int secondElemIndx = firstElemIndx + stepSize;
			vector_[firstElemIndx] += vector_[secondElemIndx];						
		}
		stepSize = stepSize * 2;
		numberOfThreads = numberOfThreads / 2;
	}
}

__global__ void findVariance (double *vector_, double mean) {
  
  int threadId = threadIdx.x;
  //printf("\nthreaad : %d, vector_[threadid] : %lf", threadId, vector_[threadId]);
  vector_[threadId] = pow((vector_[threadId] - mean), 2);
}

int main (void) {
	{
	//program for finding minimum element from input vector
	long long *hostVector, resultMinElem;
	long long *deviceVector;
	long long memorySize = VECTOR_SIZE * sizeof(long long);
	
	// Allocate space for host vectors A and insert input values
    hostVector = (long long *)malloc(memorySize); 
	fillVector(hostVector);
	
	// Allocate space for device vectors A
    cudaMalloc((void **)&deviceVector, memorySize);
	
	// Copy vector data from host to device
    cudaMemcpy(deviceVector, hostVector, memorySize, cudaMemcpyHostToDevice);
	
	
	dim3 blocksPerGrid(1, 1, 1);
	//by creating single block with half of the size of vector threads.
	dim3 threadsPerBlock(VECTOR_SIZE / 2, 1, 1);
	findMinElem<<<blocksPerGrid, threadsPerBlock>>>(deviceVector);
	
	// Copy result back to host result variable 
    cudaMemcpy(&resultMinElem, deviceVector, sizeof(long long), cudaMemcpyDeviceToHost);
	
	printf("\nInput Vector : ");
	printVector(hostVector);
	printf("\nThe minimum element in given vector : %lld", resultMinElem);
	}
	{
	//program for finding maximum element from input vector

	long long *hostVector, resultMaxElem;
	long long *deviceVector;
	long long memorySize = VECTOR_SIZE * sizeof(long long);
	
	// Allocate space for host vectors A and insert input values
    hostVector = (long long *)malloc(memorySize); 
	fillVector(hostVector);
	
	// Allocate space for device vectors A
    cudaMalloc((void **)&deviceVector, memorySize);
	
	// Copy vector data from host to device
    cudaMemcpy(deviceVector, hostVector, memorySize, cudaMemcpyHostToDevice);
	
	
	dim3 blocksPerGrid(1, 1, 1);
	//by creating single block with half of the size of vector threads.
	dim3 threadsPerBlock(VECTOR_SIZE / 2, 1, 1);
	findMaxElem<<<blocksPerGrid, threadsPerBlock>>>(deviceVector);
	
	// Copy result back to host result variable 
    cudaMemcpy(&resultMaxElem, deviceVector, sizeof(long long), cudaMemcpyDeviceToHost);
	
	printf("\nInput Vector : ");
	printVector(hostVector);
	printf("\nThe maximum element in given vector : %lld", resultMaxElem);
	}
	{
	//program for finding mean from input vector

	double *hostVector;
	double resultMean;
	double *deviceVector;
	double memorySize = VECTOR_SIZE * sizeof(double);
	
	// Allocate space for host vectors A and insert input values
    hostVector = (double *)malloc(memorySize); 
	fillVectorWDV(hostVector);
	
	// Allocate space for device vectors A
    cudaMalloc((void **)&deviceVector, memorySize);
	
	// Copy vector data from host to device
    cudaMemcpy(deviceVector, hostVector, memorySize, cudaMemcpyHostToDevice);
	
	
	dim3 blocksPerGrid(1, 1, 1);
	//by creating single block with half of the size of vector threads.
	dim3 threadsPerBlock(VECTOR_SIZE / 2, 1, 1);
	findMean<<<blocksPerGrid, threadsPerBlock>>>(deviceVector);
	
	// Copy result back to host result variable 
    cudaMemcpy(&resultMean, deviceVector, sizeof(double), cudaMemcpyDeviceToHost);
	
	printf("\nThe mean of the given vector : %lf", resultMean);
	}
	{
	//program for finding sum of all elements from input vector

	long long *hostVector, resultSum;
	long long *deviceVector;
	long long memorySize = VECTOR_SIZE * sizeof(long long);
	
	// Allocate space for host vectors A and insert input values
    hostVector = (long long *)malloc(memorySize); 
	fillVector(hostVector);
	
	// Allocate space for device vectors A
    cudaMalloc((void **)&deviceVector, memorySize);
	
	// Copy vector data from host to device
    cudaMemcpy(deviceVector, hostVector, memorySize, cudaMemcpyHostToDevice);
	
	
	dim3 blocksPerGrid(1, 1, 1);
	//by creating single block with half of the size of vector threads.
	dim3 threadsPerBlock(VECTOR_SIZE / 2, 1, 1);
	findSum<<<blocksPerGrid, threadsPerBlock>>>(deviceVector);
	
	// Copy result back to host result variable 
    cudaMemcpy(&resultSum, deviceVector, sizeof(long long), cudaMemcpyDeviceToHost);
	
	printf("\nThe summation of all the elements in given vector : %lld", resultSum);
	}
	{
	//program for finding standard deviation of elements from input vector
	//first find mean and then substract it from each element parallelly

	//program for finding mean from input vector

	double *hostVector;
	double resultMean;
	double *deviceVector;
	double memorySize = VECTOR_SIZE * sizeof(double);
	
	// Allocate space for host vectors A and insert input values
    hostVector = (double *)malloc(memorySize); 
	fillVectorWDV(hostVector);
	
	// Allocate space for device vectors A
    cudaMalloc((void **)&deviceVector, memorySize);
	
	// Copy vector data from host to device
    cudaMemcpy(deviceVector, hostVector, memorySize, cudaMemcpyHostToDevice);
	
	
	dim3 blocksPerGrid(1, 1, 1);
	//by creating single block with half of the size of vector threads.
	dim3 threadsPerBlock(VECTOR_SIZE / 2, 1, 1);
	findMean<<<blocksPerGrid, threadsPerBlock>>>(deviceVector);
	
	// Copy result back to host result variable 
    cudaMemcpy(&resultMean, deviceVector, sizeof(double), cudaMemcpyDeviceToHost);
	cudaFree(deviceVector);
	
	//again,...
	// Allocate space for device vectors A
  cudaMalloc((void **)&deviceVector, memorySize);
	double *diffSumSqrVector;
	diffSumSqrVector = (double *)malloc(memorySize); 
	
	// Copy vector data from host to device
    cudaMemcpy(deviceVector, hostVector, memorySize, cudaMemcpyHostToDevice);
	
	findVariance<<<1, VECTOR_SIZE>>>(deviceVector, resultMean);
	//diffSumSqrVector stores (x(i) - xmean)^2 at each index, further we have to sum up aall and divide with VECTOR_SIZE to get variance.
  cudaMemcpy(diffSumSqrVector, deviceVector, memorySize, cudaMemcpyDeviceToHost);
  cudaFree(deviceVector);

  double resultVarSum;
  //again,...
	// Allocate space for device vectors A
  cudaMalloc((void **)&deviceVector, memorySize);
	
	// Copy vector data from host to device
    cudaMemcpy(deviceVector, diffSumSqrVector, memorySize, cudaMemcpyHostToDevice);
	
	findSumVar<<<1, VECTOR_SIZE>>>(deviceVector);
  // Copy result back to host result variable 
  cudaMemcpy(&resultVarSum, deviceVector, sizeof(double), cudaMemcpyDeviceToHost);
  cudaFree(deviceVector);

  double variance = resultVarSum / (double) (VECTOR_SIZE - 1);

  printf("\nThe Variance of the elements in given vector : %lf", variance);

  double standardDeviation = sqrt(variance);

  printf("\nThe Standard Deviation of the elements in given vector : %lf", standardDeviation);

	}
  	return 0;
}


Input Vector : 1 2 3 4 5 6 7 8 
The minimum element in given vector : 1
Input Vector : 1 2 3 4 5 6 7 8 
The maximum element in given vector : 8
The mean of the given vector : 4.500000
The summation of all the elements in given vector : 36
The Variance of the elements in given vector : 6.000000
The Standard Deviation of the elements in given vector : 2.449490
